# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [ ]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

In [ ]:
A

array([[1.51915195e+01, 6.22108771e-01, 4.37727739e-01, 7.85358584e-01,
        7.79975808e-01, 2.72592605e-01, 2.76464255e-01, 8.01872178e-01,
        9.58139354e-01, 8.75932635e-01],
       [3.57817270e-01, 1.55009951e+01, 6.83462935e-01, 7.12702027e-01,
        3.70250755e-01, 5.61196186e-01, 5.03083165e-01, 1.37684496e-02,
        7.72826622e-01, 8.82641191e-01],
       [3.64885984e-01, 6.15396178e-01, 1.50753812e+01, 3.68824006e-01,
        9.33140102e-01, 6.51378143e-01, 3.97202578e-01, 7.88730143e-01,
        3.16836122e-01, 5.68098653e-01],
       [8.69127390e-01, 4.36173424e-01, 8.02147642e-01, 1.51437668e+01,
        7.04260971e-01, 7.04581308e-01, 2.18792106e-01, 9.24867629e-01,
        4.42140755e-01, 9.09315959e-01],
       [5.98092228e-02, 1.84287084e-01, 4.73552788e-02, 6.74880944e-01,
        1.55946248e+01, 5.33310163e-01, 4.33240627e-02, 5.61433080e-01,
        3.29668446e-01, 5.02966833e-01],
       [1.11894318e-01, 6.07193706e-01, 5.65944643e-01, 6.76406199e-03,
   

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where 

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [ ]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B 
c = invD @ b

In [ ]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [ ]:
np.linalg.norm(BB)

0.31853784252598116

### Do the Jacobi iteration

In [ ]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [ ]:
# Check the result:

A @ x - b

array([  1.11022302e-16,   1.11022302e-16,  -1.11022302e-16,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
        -1.38777878e-17,   0.00000000e+00,   2.77555756e-17,
         1.11022302e-16])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction. 
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [4]:
import numpy as np

def jacobi_iteration(A, b, num_iterations):
    n = len(b)
    x = np.zeros(n)  # Initial guess for x
    D = np.diag(np.diag(A))  # Diagonal matrix of A

    for _ in range(num_iterations):
        x_new = (b - np.dot(A, x) + np.dot(D, x)) / np.diag(D)
        x = x_new

    return x

In [9]:
n = 5  # Size of the matrix
A = np.random.rand(n, n)  # Random matrix
b = np.random.rand(n)  # Random right-hand side vector
num_iterations = 10

num_iterations = 10

x = jacobi_iteration(A, b, num_iterations)

# Calculate the residual vector r
r = b - np.dot(A, x)

# Calculate the norm of the residual vector
residual_norm = np.linalg.norm(r)

print("Solution x:", x)
print("Norm of the residual vector:", residual_norm)

Solution x: [-2.27622042e+08 -4.57554907e+07 -5.66636847e+07 -5.64885254e+07
 -4.83821137e+07]
Norm of the residual vector: 618640788.5949675


# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration. 

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [7]:
import numpy as np

def seidel_iteration(A, b, num_iterations):
    n = len(b)
    x = np.zeros(n)  # Initial guess for x

    for _ in range(num_iterations):
        for i in range(n):
            x[i] = (b[i] - np.dot(A[i, :i], x[:i]) - np.dot(A[i, i+1:], x[i+1:])) / A[i, i]

    return x

In [10]:

n = 5  # Size of the matrix
A = np.random.rand(n, n)  # Random matrix
b = np.random.rand(n)  # Random right-hand side vector
num_iterations = 10

x = seidel_iteration(A, b, num_iterations)

# Calculate the residual vector r
r = b - np.dot(A, x)

# Calculate the norm of the residual vector
residual_norm = np.linalg.norm(r)

# Calculate the iteration matrix
iteration_matrix = np.eye(n) - np.dot(np.linalg.inv(A), A)

# Calculate the norm of the iteration matrix
iteration_matrix_norm = np.linalg.norm(iteration_matrix)

print("Solution x:", x)
print("Norm of the residual vector:", residual_norm)
print("Norm of the iteration matrix:", iteration_matrix_norm)

Solution x: [-23.16089592  11.12910839  39.52854964 -16.35266047 -34.24835819]
Norm of the residual vector: 17.399599975656333
Norm of the iteration matrix: 1.7804223964965056e-15


# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [12]:
import numpy as np

def minres_method(A, b, num_iterations):
    n = len(b)
    x = np.zeros(n)  # Initial guess for x
    r = b - np.dot(A, x)
    p = r
    alpha = np.dot(r, r)
    beta = 0

    for _ in range(num_iterations):
        Ap = np.dot(A, p)
        denom = np.dot(p, Ap)
        if denom == 0:
            break
        tau = alpha / denom
        x = x + tau * p
        r = r - tau * Ap
        alpha_new = np.dot(r, r)
        beta = alpha_new / alpha
        p = r + beta * p
        alpha = alpha_new

    return x

In [13]:
n = 5  # Size of the matrix
A = np.random.rand(n, n)  # Random matrix
b = np.random.rand(n)  # Random right-hand side vector
num_iterations = 10

# Solve the system using the MINRES method
x_minres = minres_method(A, b, num_iterations)

# Solve the system using a direct method (ground truth solution)
x_direct = np.linalg.solve(A, b)

# Calculate the residual vector r
r = b - np.dot(A, x_minres)

# Calculate the norm of the residual vector
residual_norm = np.linalg.norm(r)

# Calculate the deviation from the ground truth solution
deviation = np.linalg.norm(x_minres - x_direct)

print("Solution (MINRES method):", x_minres)
print("Solution (Direct method):", x_direct)
print("Norm of the residual vector:", residual_norm)
print("Deviation from the ground truth solution:", deviation)

Solution (MINRES method): [-30.31327174  72.02314746 -30.95465817  71.65114199  25.06578574]
Solution (Direct method): [ 1.03447993 -1.20284012  1.41095229 -0.99591155 -0.09396147]
Norm of the residual vector: 69.51184903283928
Deviation from the ground truth solution: 115.33805442803893
